### Imports

In [76]:
import pandas as pd
import numpy as np
import glob
import copy
import os

### Read the thresholds table

In [77]:
thresholds = pd.read_csv("cutpoint_values - thresholds.csv", index_col=0)
# replace - with nan
thresholds.replace({"-": np.nan}, inplace=True)

### Read cohorts

In [78]:
# datasets = [pd.read_csv(file, index_col=0, low_memory=False) for file in sorted(glob.glob('../../cohort_studies_full_data/' + "/*."+'csv'))]
# cohorts = [file.split(".")[0] for file in sorted(os.listdir('../../cohort_studies_full_data/'))]

In [79]:
# # make a dictionary that contains all cohorts as a dataframe
# cohort_studies = dict()
# # dfsss = dict()

# for cohort, dataset in zip(cohorts, datasets):
#     cohort_n = cohort.split("_MERGE")[0]
#     cohort_studies[cohort_n] = dataset.loc[dataset['Months']==0].copy() # reduce to BL visit

In [80]:
# select_coh = dict()

# for i in thresholds.index:
#     if i=="NACC_ELISA" or i=="NACC_XMAP":
#         select_coh['NACC'] = cohort_studies['NACC']
        
#     elif i=="EMIF_ELISA" or i=="EMIF_XMAP":
#         select_coh['EMIF'] = cohort_studies['EMIF']
        
#     else:
#         select_coh[i] = cohort_studies[i]

### Read mapping tables

In [81]:
# modality = [pd.read_csv(file, sep=',') for file in sorted(glob.glob('../../feature_tables' + "/*."+'csv'))]
# name = [file.split(".")[0] for file in sorted(os.listdir('../../feature_tables'))]

In [82]:
# # make a dictionary that contains all modalities as a dataframe
# mappings = dict()

# for moda, na in zip(modality, name):
#     mappings[na.split(' - ')[1]] = moda

### select patients with CSF measurements

In [49]:
# for i in select_coh:
#     select_coh[i].dropna(subset=mappings['csf'][i].iloc[:3].to_list(), inplace=True)

In [54]:
# # divide patients with different assays 
# select_coh['NACC_ELISA'] = select_coh['NACC'].loc[select_coh[i]['CSFTTMD']==1]
# select_coh['NACC_XMAP'] = select_coh['NACC'].loc[select_coh[i]['CSFTTMD']==2]
# select_coh['EMIF_ELISA'] = select_coh['EMIF'].loc[~(select_coh['EMIF']['Studyname'].isin(['EDAR', 'Leuven']))]
# select_coh['EMIF_XMAP'] = select_coh['EMIF'].loc[(select_coh['EMIF']['Studyname'].isin(['EDAR', 'Leuven']))]

In [58]:
# for i in select_coh:
#     print(i, len(select_coh[i].index))

ADNI 1017
EPAD 1237
AIBL 57
ARWIBO 217
EDSD 86
PharmaCog 145
PREVENT-AD 133
NACC 506
EMIF 1014
DOD-ADNI 113
JADNI 197
NACC_ELISA 205
NACC_XMAP 301
EMIF_ELISA 811
EMIF_XMAP 203


### Calculate the relative change amon thresholds for each biomarker in each cohort

In [91]:
# change the type of values into float and round the values by one decimal
thresholds = thresholds.astype(float).round(1)
# make a df to store the results
change_df = pd.DataFrame(index=thresholds.index, columns=['Aβ1-42', 'pTau', 'tTau'])
change_max = pd.DataFrame(index=thresholds.index, columns=['Aβ1-42', 'pTau', 'tTau'])

# for each cohort dataset check which methods were performed and make a list of method for each biomarker
for i in thresholds.index:
    ab, pt, tt = [], [], []
    df = pd.DataFrame(thresholds.loc[i]).transpose().dropna(axis=1)

    for col in df.columns:
        if "Aβ" in col: 
            ab.append(col)

        elif 'pTau' in col:
            pt.append(col)

        else:
            tt.append(col)
                   
    # find the maximum threshold for abeta 
    max_ab = float(df.loc[i, df[ab].astype(float).idxmax(axis=1)].item())
    # find the minimum threshold for abeta
    min_ab = float(df.loc[i, df[ab].astype(float).idxmin(axis=1)].item()) 
    # calculate the relative change (max - min / max) among abeta thresholds
    change_df.loc[i, 'Aβ1-42'] = round(((max_ab - min_ab) / max_ab) * 100, 2) 
    change_max.loc[i, 'Aβ1-42'] = max_ab
    
    # find the maximum threshold for ptau 
    max_pt = float(df.loc[i, df[pt].astype(float).idxmax(axis=1)].item())
    # find the minimum threshold for ptau
    min_pt = float(df.loc[i, df[pt].astype(float).idxmin(axis=1)].item())
    # calculate the relative change (max - min / max) among ptau thresholds
    change_df.loc[i, 'pTau'] = round(((max_pt - min_pt) / max_pt) * 100, 2)  
    change_max.loc[i, 'pTau'] = max_pt
    
    # find the maximum threshold for ttau 
    max_tt = float(df.loc[i, df[tt].astype(float).idxmax(axis=1)].item())
    # find the minimum threshold for ttau
    min_tt = float(df.loc[i, df[tt].astype(float).idxmin(axis=1)].item())
    # calculate the relative change (max - min / max) among ttau thresholds
    change_df.loc[i, 'tTau'] = round(((max_tt - min_tt) / max_tt) * 100, 2)
    change_max.loc[i, 'tTau'] = max_tt

In [92]:
tt

['tTau_GMM', 'tTau_K-Means', 'tTau_Tertile', 'tTau_ROC', 'tTau_Mean ±2 SD']

In [93]:
change_df

,Aβ1-42,pTau,tTau
Cohort,,,
ADNI,71.45,47.43,41.3
EPAD,67.35,54.27,51.01
AIBL,3.43,16.09,6.89
ARWIBO,11.2,45.17,18.6
EDSD,6.74,26.86,7.27
PharmaCog,0.77,28.89,26.13
PREVENT-AD,48.85,43.86,49.57
NACC_ELISA,53.66,45.45,48.33
EMIF_ELISA,76.32,45.22,59.76


In [96]:
change_df_subset = change_df.drop(index=['AIBL', 'ARWIBO', 'EDSD', 'PharmaCog'])
change_max_subset = change_max.drop(index=['AIBL', 'ARWIBO', 'EDSD', 'PharmaCog'])

In [88]:
change_df_subset.loc['Average'] = round(change_df_subset.mean(), 2)

In [90]:
change_df_subset.to_csv("relative_change.csv")

In [89]:
change_df_subset

,Aβ1-42,pTau,tTau
Cohort,,,
ADNI,71.45,47.43,41.3
EPAD,67.35,54.27,51.01
PREVENT-AD,48.85,43.86,49.57
NACC_ELISA,53.66,45.45,48.33
EMIF_ELISA,76.32,45.22,59.76
NACC_XMAP,92.7,47.62,46.04
EMIF_XMAP,46.8,48.01,59.26
DOD-ADNI,77.87,42.27,38.09
JADNI,55.12,41.24,50.48


In [97]:
change_max_subset

,Aβ1-42,pTau,tTau
Cohort,,,
ADNI,983.7,40.9,409.7
EPAD,1034.9,43.3,436.4
PREVENT-AD,1150.4,89.6,598.1
NACC_ELISA,629,88,728.8
EMIF_ELISA,730.9,95.1,669.4
NACC_XMAP,298.6,69.3,99.7
EMIF_XMAP,450.2,55.2,204.2
DOD-ADNI,1276,34.3,369.9
JADNI,416.4,66.2,145.8


In [72]:
# # Roche
# np.mean(list(round(change_df.loc[['ADNI', 'EPAD']].mean(), 2)))

55.47

In [69]:
# # INNOTEST
# np.mean(round(change_df.loc[['AIBL', 'ARWIBO', 'EDSD', 'PharmaCog', 'PREVENT-AD', 'NACC_ELISA', 'EMIF_ELISA']].mean(), 2))

31.86

In [70]:
# # xMAP
# np.mean(round(change_df.loc[['NACC_XMAP', 'EMIF_XMAP', 'DOD-ADNI', 'JADNI']].mean(), 2))

53.79

In [76]:
# change_df.loc['Average'].mean()

42.24

In [7]:
# thresholds.replace({np.nan: "-"}).to_csv("thresholds_one_decimal.csv")

In [9]:
thresholds

,Aβ1-42_GMM,pTau_GMM,tTau_GMM,Aβ1-42_K-Means,pTau_K-Means,tTau_K-Means,Aβ1-42_Tertile,pTau_Tertile,tTau_Tertile,Aβ1-42_ROC,pTau_ROC,tTau_ROC,Aβ1-42_Mean ±2 SD,pTau_Mean ±2 SD,tTau_Mean ±2 SD
Cohort,,,,,,,,,,,,,,,
ADNI,983.7,34.3,352.3,973.0,28.1,286.2,832.1,21.5,240.5,801.7,24.2,264.8,280.8,40.9,409.7
EPAD,1034.9,27.7,306.7,1030.7,19.8,223.7,883.0,19.9,228.3,731.5,20.9,213.8,337.9,43.3,436.4
AIBL,714.4,92.6,591.5,689.9,77.7,635.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ARWIBO,577.5,126.2,499.0,512.8,69.2,406.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EDSD,737.5,113.2,535.7,790.8,82.8,577.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PharmaCog,779.6,93.1,630.6,773.6,66.2,465.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PREVENT-AD,1030.9,68.0,424.8,1150.4,50.3,301.6,1104.4,53.1,304.2,NaN,NaN,NaN,588.4,89.6,598.1
NACC_ELISA,596.2,80.8,537.4,591.4,61.1,497.4,629.0,48.0,376.6,474.0,64.0,468.0,291.5,88.0,728.8
EMIF_ELISA,730.9,81.4,523.8,723.7,65.0,435.1,538.2,52.1,269.4,560.0,59.0,355.4,173.1,95.1,669.4


In [12]:
thresholds.loc[['ADNI', 'EPAD'], 'Aβ1-42_GMM'].max()

1034.9

In [36]:
changes_cross_cohorts = pd.DataFrame(index=['group1', 'group2', 'group3'], columns=thresholds.columns)
changes_cross_cohorts.index.name = "groups"

In [57]:
def changes_among_cohorts(cohorts, g, result_):
    "Relative changes among the thresholds obtained using each method within cohorts with the same assay"

    for i in thresholds.columns:
        up, low = thresholds.loc[cohorts, i].max(), thresholds.loc[cohorts, i].min()
        result_.loc[g, i] = round(((up - low) / up )* 100, 2)

In [58]:
changes_among_cohorts(['ADNI', 'EPAD'], 'group1', changes_cross_cohorts)
changes_among_cohorts(['AIBL', 'ARWIBO', 'EDSD', 'PharmaCog', 'PREVENT-AD', 'NACC_ELISA', 'EMIF_ELISA'], 'group2', changes_cross_cohorts)
changes_among_cohorts(['NACC_XMAP', 'EMIF_XMAP', 'DOD-ADNI', 'JADNI'], 'group3', changes_cross_cohorts)

In [59]:
changes_cross_cohorts

,Aβ1-42_GMM,pTau_GMM,tTau_GMM,Aβ1-42_K-Means,pTau_K-Means,tTau_K-Means,Aβ1-42_Tertile,pTau_Tertile,tTau_Tertile,Aβ1-42_ROC,pTau_ROC,tTau_ROC,Aβ1-42_Mean ±2 SD,pTau_Mean ±2 SD,tTau_Mean ±2 SD
groups,,,,,,,,,,,,,,,
group1,4.95,19.24,12.94,5.6,29.54,21.84,5.76,7.44,5.07,8.76,13.64,19.26,16.9,5.54,6.12
group2,43.98,46.12,32.64,55.42,39.25,52.53,51.27,9.6,28.47,15.36,7.81,24.06,70.58,7.47,17.93
group3,76.6,56.77,69.29,78.06,63.71,72.02,73.59,49.1,76.51,35.72,36.08,43.36,92.28,50.51,73.05


In [95]:
changes_cross_cohorts.loc['group1', ab]

Aβ1-42_GMM           4.95
Aβ1-42_K-Means        5.6
Aβ1-42_Tertile       5.76
Aβ1-42_ROC           8.76
Aβ1-42_Mean ±2 SD    16.9
Name: group1, dtype: object

In [96]:
changes_cross_cohorts.loc['group1', ab].mean()

8.394

In [61]:
cross_method = pd.DataFrame(index=changes_cross_cohorts.index, columns=['GMM', 'K-Means', 'Tertile', 'ROC', 'Mean ±2 SD'])
cross_biomarker = pd.DataFrame(index=changes_cross_cohorts.index, columns= ['Aβ1-42', 'pTau', 'tTau'])

In [73]:
def average_among_biomarker(result_df, g):
    """Take the average relative changes for each biomarker coming from different methods among cohorts with the same assay"""

    for i in changes_cross_cohorts.index:
        result_df.loc[g, 'Aβ1-42'] = round(changes_cross_cohorts.loc[g, ab].mean(), 2)
        result_df.loc[g, 'pTau'] = round(changes_cross_cohorts.loc[g, pt].mean(), 2)
        result_df.loc[g, 'tTau'] = round(changes_cross_cohorts.loc[g, tt].mean(), 2)

In [74]:
average_among_biomarker(cross_biomarker, 'group1')
average_among_biomarker(cross_biomarker, 'group2')
average_among_biomarker(cross_biomarker, 'group3')

In [75]:
cross_biomarker

,Aβ1-42,pTau,tTau
groups,,,
group1,8.39,15.08,13.05
group2,47.32,22.05,31.13
group3,71.25,51.23,66.85


In [84]:
print(changes_cross_cohorts.columns.to_list())

['Aβ1-42_GMM', 'pTau_GMM', 'tTau_GMM', 'Aβ1-42_K-Means', 'pTau_K-Means', 'tTau_K-Means', 'Aβ1-42_Tertile', 'pTau_Tertile', 'tTau_Tertile', 'Aβ1-42_ROC', 'pTau_ROC', 'tTau_ROC', 'Aβ1-42_Mean ±2 SD', 'pTau_Mean ±2 SD', 'tTau_Mean ±2 SD']


In [85]:
def average_cross_method(g):
    """Take the average relative changes within thresholds of each method among cohorts with the same assay"""
    cross_method.loc[g, 'GMM'] = round(changes_cross_cohorts.loc[g, ['Aβ1-42_GMM', 'pTau_GMM', 'tTau_GMM']].mean(), 2)
    cross_method.loc[g, 'K-Means'] = round(changes_cross_cohorts.loc[g, ['Aβ1-42_K-Means', 'pTau_K-Means', 'tTau_K-Means']].mean(), 2)
    cross_method.loc[g, 'Tertile'] = round(changes_cross_cohorts.loc[g, ['Aβ1-42_Tertile', 'pTau_Tertile', 'tTau_Tertile']].mean(), 2)
    cross_method.loc[g, 'ROC'] = round(changes_cross_cohorts.loc[g, ['Aβ1-42_ROC', 'pTau_ROC', 'tTau_ROC']].mean(), 2)
    cross_method.loc[g, 'Mean ±2 SD'] = round(changes_cross_cohorts.loc[g, ['Aβ1-42_Mean ±2 SD', 'pTau_Mean ±2 SD', 'tTau_Mean ±2 SD']].mean(), 2)


In [86]:
average_cross_method('group1')
average_cross_method('group2')
average_cross_method('group3')

In [88]:
cross_method

,GMM,K-Means,Tertile,ROC,Mean ±2 SD
groups,,,,,
group1,12.38,18.99,6.09,13.89,9.52
group2,40.91,49.07,29.78,15.74,31.99
group3,67.55,71.26,66.4,38.39,71.95
